In [1]:
import torch

In [11]:
import pyro

In [2]:
import pyro.distributions as dist

In [3]:
from torch_geometric.utils import stochastic_blockmodel_graph, get_laplacian

In [4]:
from torch_geometric.utils import to_networkx

In [5]:
from torch_geometric.data import Data

In [6]:
import networkx as nx
import matplotlib.pyplot as plt

In [239]:
pyro.clear_param_store()
num_nodes = 127

n_blocks_logits = pyro.param("n_block_logits", torch.randn(num_nodes))
n_blocks_distrib = dist.Categorical(logits=n_block_logits)
n_blocks = pyro.sample("n_blocks", n_blocks_distrib)

block_assignments = []
for i in range(num_sensors):
    assignment_logits = pyro.param(f"assigment_logits_{i}", torch.randn(n_blocks))
    assignment_dist = dist.Categorical(logits=assignment_logits)
    
    block_assignments.append(
        pyro.sample(f"block_assignment_{i}", assignment_dist)
    )    
block_assignments = torch.Tensor(block_assignments)

block_sizes = []
for i in range(n_blocks):
    block_sizes.append(
        (block_assignments == i).sum()
    )
block_sizes = torch.Tensor(block_sizes).long()

In [7]:
from torch_geometric.datasets import KarateClub

In [118]:
n_blocks = 4
min_size = 5
max_size = 10

block_sizes = torch.randint(min_size, max_size, size=(n_blocks,))
block_probs = torch.rand(n_blocks, n_blocks)
edge_index = stochastic_blockmodel_graph(
    block_sizes,
    block_probs,
    directed=True
)
#data = Data(x=torch.randn(block_sizes.sum()), edge_index=edge_index)
#G = to_networkx(data)

#plt.figure(figsize=(10,10), dpi=200)
#nx.draw_networkx(G, pos=nx.spectral_layout(G), with_labels=False, node_size=5, width=0.1, arrows=False)

In [119]:
edge_index.shape

torch.Size([2, 362])

In [120]:
kc = KarateClub().get(0)

In [121]:
kc.edge_index.shape

torch.Size([2, 156])